In [20]:
import tensorflow as tf
from tensorflow import keras as keras
from keras.layers import Flatten, Dense
from keras.layers import Input
import numpy as np

In [33]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
TOTAL_IMAGE_COUNT = 8
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 3
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 50
IMAGE_HEIGHT = 50

BATCH_SIZE = 2*NUM_VARIETIES
LEARNING_RATE_BASE = 0.0001
NUM_EPOCHS = 40

In [22]:
def dataset_file_name():
    return "./dataset/V"+str(NUM_VARIETIES).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)

In [23]:
DATASET_FILE_NAME = dataset_file_name()
train_dataset = np.load(DATASET_FILE_NAME+"_train_dataset.npy")
train_dataset_label = np.load(DATASET_FILE_NAME+"_train_dataset_label.npy")
test_dataset = np.load(DATASET_FILE_NAME+"_test_dataset.npy")
test_dataset_label = np.load(DATASET_FILE_NAME+"_test_dataset_label.npy")

In [24]:
model = keras.models.Sequential()
inp = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_OF_BANDS))

In [25]:
denseNet = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=inp,
    input_shape=None,
    pooling=None,
    classes=NUM_VARIETIES,
    classifier_activation="softmax",
)

In [26]:
model.add(denseNet)
model.add(Flatten())
model.add(Dense(10))

In [27]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1, 1, 1024)        7037504   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 7,047,754
Trainable params: 6,964,106
Non-trainable params: 83,648
_________________________________________________________________


In [28]:
import timeit
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [29]:
from keras.optimizers import Adam
adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

In [ ]:
tic = start_timer()
model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, initial_epoch = 0, validation_split=0.2, verbose=2,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/40
2/2 - 8s - loss: 3.3933 - accuracy: 0.5000 - val_loss: 6.4697 - val_accuracy: 0.0000e+00 - 8s/epoch - 4s/step
Epoch 2/40
2/2 - 1s - loss: 1.6359 - accuracy: 0.8333 - val_loss: 4.3363 - val_accuracy: 0.0000e+00 - 536ms/epoch - 268ms/step
Epoch 3/40
2/2 - 1s - loss: 3.5021 - accuracy: 0.5000 - val_loss: 3.9876 - val_accuracy: 0.2500 - 568ms/epoch - 284ms/step
Epoch 4/40
2/2 - 1s - loss: 3.2073 - accuracy: 0.4167 - val_loss: 1.6054 - val_accuracy: 0.2500 - 537ms/epoch - 268ms/step
Epoch 5/40
2/2 - 1s - loss: 3.1014 - accuracy: 0.6667 - val_loss: 1.5978 - val_accuracy: 0.2500 - 551ms/epoch - 275ms/step
Epoch 6/40
2/2 - 1s - loss: 2.0951 - accuracy: 0.6667 - val_loss: 1.5918 - val_accuracy: 0.2500 - 554ms/epoch - 277ms/step
Epoch 7/40
2/2 - 1s - loss: 3.4245 - accuracy: 0.5000 - val_loss: 1.5989 - val_accuracy: 0.2500 - 531ms/epoch - 266ms/step
Epoch 8/40
2/2 - 1s - loss: 2.1739 - accuracy: 0.5833 - val_loss: 1.6074 - val_accuracy: 0.2500 - 573ms/epoch - 287ms/ste

In [31]:
print("---Train Accuracy---")
model.evaluate(train_dataset,train_dataset_label)
print("---Test Accuracy---")
model.evaluate(test_dataset,test_dataset_label)

---Train Accuracy---
1/1 [==============================] - 2s 2s/step - loss: 8.0915 - accuracy: 0.1875
---Test Accuracy---
1/1 [==============================] - 0s 115ms/step - loss: 10.3611 - accuracy: 0.1250


[10.361146926879883, 0.125]